# Linear vs Non-Linear Mixing: A Simulation Study

This notebook illustrates the mathematical and practical differences between:

1. **Linear Mixing** (Aladynoulli approach): `π = sigmoid(Σₖ softmax(λₖ) × sigmoid(φₖ))`
   - Each signature is converted to probability scale, then mixed
   - Better identifiability, normalized mixing weights

2. **Non-Linear Mixing** (Alternative): `π = sigmoid(Σₖ λₖ × φₖ)`
   - Raw mixing before probability transformation
   - Better separation potential, but scale invariance issues

## Key Questions

- How do they differ in disease separation?
- What about scale invariance?
- Which preserves signal differences better?


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import expit as sigmoid
from scipy.special import softmax

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
np.random.seed(42)


## 1. Mathematical Formulation

Let's define both approaches clearly:


In [ ]:
def linear_mixing(lambda_vals, phi_vals):
    """
    Linear mixing: sigmoid(Σ softmax(λ) × sigmoid(φ))
    Each signature converted to prob scale, then mixed
    """
    lambda_probs = softmax(lambda_vals)  # Normalized mixing weights
    phi_probs = sigmoid(phi_vals)  # Each signature to probability
    mixed = np.sum(lambda_probs * phi_probs, axis=-1)
    return sigmoid(mixed)

def nonlinear_mixing(lambda_vals, phi_vals):
    """
    Non-linear mixing: sigmoid(Σ λ × φ)
    Raw mixing before probability transformation
    """
    raw_mixed = np.sum(lambda_vals * phi_vals, axis=-1)
    return sigmoid(raw_mixed)


## 2. Disease Separation: Can We Distinguish Two Diseases?

**Scenario**: Two diseases with overlapping signatures but different loadings.

- Disease A: High loading on Signature 1, low on Signature 2
- Disease B: Low loading on Signature 1, high on Signature 2

Question: Which mixing approach better separates these diseases?


In [ ]:
# Simulate two diseases with different signature loadings
n_samples = 1000
n_signatures = 3

# Disease A: Strong signature 1, weak signature 2
lambda_A = np.array([3.0, 0.5, 1.0])  # High, low, medium

# Disease B: Weak signature 1, strong signature 2
lambda_B = np.array([0.5, 3.0, 1.0])  # Low, high, medium

# Shared signatures (same for both diseases)
phi = np.array([2.0, 2.0, 1.5])  # Signature strengths

# Generate probabilities for both diseases
pi_A_linear = linear_mixing(lambda_A, phi)
pi_B_linear = linear_mixing(lambda_B, phi)

pi_A_nonlinear = nonlinear_mixing(lambda_A, phi)
pi_B_nonlinear = nonlinear_mixing(lambda_B, phi)

print(f"Linear Mixing:")
print(f"  Disease A probability: {pi_A_linear:.4f}")
print(f"  Disease B probability: {pi_B_linear:.4f}")
print(f"  Separation (|A - B|): {abs(pi_A_linear - pi_B_linear):.4f}")
print(f"\nNon-Linear Mixing:")
print(f"  Disease A probability: {pi_A_nonlinear:.4f}")
print(f"  Disease B probability: {pi_B_nonlinear:.4f}")
print(f"  Separation (|A - B|): {abs(pi_A_nonlinear - pi_B_nonlinear):.4f}")


In [ ]:
# Visualize separation across a range of lambda values
lambda1_range = np.linspace(0.1, 5, 50)
lambda2_range = np.linspace(0.1, 5, 50)

L1, L2 = np.meshgrid(lambda1_range, lambda2_range)

# Fix lambda3 = 1.0
lambda3_fixed = 1.0

# Compute probabilities for both approaches
pi_linear = np.zeros_like(L1)
pi_nonlinear = np.zeros_like(L1)

for i in range(L1.shape[0]):
    for j in range(L1.shape[1]):
        lambda_vals = np.array([L1[i,j], L2[i,j], lambda3_fixed])
        pi_linear[i,j] = linear_mixing(lambda_vals, phi)
        pi_nonlinear[i,j] = nonlinear_mixing(lambda_vals, phi)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Linear mixing
im1 = axes[0].contourf(L1, L2, pi_linear, levels=20, cmap='viridis')
axes[0].set_xlabel('λ₁ (Signature 1 Loading)', fontsize=12)
axes[0].set_ylabel('λ₂ (Signature 2 Loading)', fontsize=12)
axes[0].set_title('Linear Mixing: π = sigmoid(Σ softmax(λ) × sigmoid(φ))', fontsize=14, fontweight='bold')
axes[0].plot(lambda_A[0], lambda_A[1], 'ro', markersize=12, label='Disease A')
axes[0].plot(lambda_B[0], lambda_B[1], 'bs', markersize=12, label='Disease B')
axes[0].legend(fontsize=10)
plt.colorbar(im1, ax=axes[0], label='Disease Probability π')

# Non-linear mixing
im2 = axes[1].contourf(L1, L2, pi_nonlinear, levels=20, cmap='viridis')
axes[1].set_xlabel('λ₁ (Signature 1 Loading)', fontsize=12)
axes[1].set_ylabel('λ₂ (Signature 2 Loading)', fontsize=12)
axes[1].set_title('Non-Linear Mixing: π = sigmoid(Σ λ × φ)', fontsize=14, fontweight='bold')
axes[1].plot(lambda_A[0], lambda_A[1], 'ro', markersize=12, label='Disease A')
axes[1].plot(lambda_B[0], lambda_B[1], 'bs', markersize=12, label='Disease B')
axes[1].legend(fontsize=10)
plt.colorbar(im2, ax=axes[1], label='Disease Probability π')

plt.tight_layout()
plt.savefig('linear_vs_nonlinear_separation.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nSeparation Analysis:")
print(f"Linear:   |π_A - π_B| = {abs(pi_A_linear - pi_B_linear):.4f}")
print(f"NonLinear: |π_A - π_B| = {abs(pi_A_nonlinear - pi_B_nonlinear):.4f}")


## 3. Scale Invariance: The Key Issue

**Problem**: With non-linear mixing, `sigmoid(λ × φ) = sigmoid((λ/c) × (φ×c))` for any constant `c`.

This means the model can't distinguish between:
- `λ=2, φ=3` → `sigmoid(6)`
- `λ=1, φ=6` → `sigmoid(6)`

**Linear mixing avoids this** because `softmax(λ)` normalizes the weights.


In [ ]:
# Demonstrate scale invariance issue
lambda_original = np.array([2.0, 1.0, 0.5])
phi_original = np.array([3.0, 2.0, 1.0])

# Scale lambda down, phi up (preserves product)
c = 2.0
lambda_scaled = lambda_original / c
phi_scaled = phi_original * c

print("Scale Invariance Test:")
print("="*60)
print(f"Original: λ = {lambda_original}, φ = {phi_original}")
print(f"Scaled:   λ = {lambda_scaled}, φ = {phi_scaled}")
print(f"\nProduct (λ × φ) preserved: {np.allclose(lambda_original * phi_original, lambda_scaled * phi_scaled)}")

# Test both approaches
pi_orig_linear = linear_mixing(lambda_original, phi_original)
pi_scaled_linear = linear_mixing(lambda_scaled, phi_scaled)

pi_orig_nonlinear = nonlinear_mixing(lambda_original, phi_original)
pi_scaled_nonlinear = nonlinear_mixing(lambda_scaled, phi_scaled)

print(f"\nLinear Mixing:")
print(f"  Original: π = {pi_orig_linear:.6f}")
print(f"  Scaled:   π = {pi_scaled_linear:.6f}")
print(f"  Same? {np.isclose(pi_orig_linear, pi_scaled_linear):.6f} (should be False for identifiability)")

print(f"\nNon-Linear Mixing:")
print(f"  Original: π = {pi_orig_nonlinear:.6f}")
print(f"  Scaled:   π = {pi_scaled_nonlinear:.6f}")
print(f"  Same? {np.isclose(pi_orig_nonlinear, pi_scaled_nonlinear):.6f} (PROBLEM: scale invariant!)")


In [ ]:
# Visualize scale invariance
scales = np.linspace(0.5, 3.0, 50)
pi_linear_vals = []
pi_nonlinear_vals = []

for c in scales:
    lambda_c = lambda_original / c
    phi_c = phi_original * c
    pi_linear_vals.append(linear_mixing(lambda_c, phi_c))
    pi_nonlinear_vals.append(nonlinear_mixing(lambda_c, phi_c))

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(scales, pi_linear_vals, 'b-', linewidth=2, label='Linear Mixing', alpha=0.8)
ax.plot(scales, pi_nonlinear_vals, 'r--', linewidth=2, label='Non-Linear Mixing', alpha=0.8)
ax.axhline(y=pi_orig_linear, color='b', linestyle=':', alpha=0.5, label='Original (Linear)')
ax.axhline(y=pi_orig_nonlinear, color='r', linestyle=':', alpha=0.5, label='Original (Non-Linear)')
ax.set_xlabel('Scale Factor c', fontsize=12)
ax.set_ylabel('Disease Probability π', fontsize=12)
ax.set_title('Scale Invariance: π(λ/c, φ×c) vs π(λ, φ)', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('scale_invariance.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nKey Insight:")
print("  Non-linear mixing is scale-invariant (flat line) → identifiability problem")
print("  Linear mixing changes with scale → better identifiability")


## 4. Signal Preservation: Before vs After Compression

**Claim**: Non-linear mixing preserves signal differences better because sigmoid compression happens *after* mixing.

Let's test this with two scenarios:
- Small difference in raw signals
- Large difference in raw signals


In [ ]:
# Test signal preservation
n_signatures = 3

# Scenario 1: Small difference
lambda_small_diff = np.array([2.0, 1.0, 1.0])
lambda_small_diff_2 = np.array([2.1, 1.0, 1.0])  # Slightly different

# Scenario 2: Large difference
lambda_large_diff = np.array([5.0, 1.0, 1.0])
lambda_large_diff_2 = np.array([5.5, 1.0, 1.0])  # Same relative difference

phi_test = np.array([2.0, 1.5, 1.0])

print("Signal Preservation Test:")
print("="*60)

# Small difference
pi_small_linear_1 = linear_mixing(lambda_small_diff, phi_test)
pi_small_linear_2 = linear_mixing(lambda_small_diff_2, phi_test)
pi_small_nonlinear_1 = nonlinear_mixing(lambda_small_diff, phi_test)
pi_small_nonlinear_2 = nonlinear_mixing(lambda_small_diff_2, phi_test)

print(f"Small Difference (λ₁: 2.0 → 2.1):")
print(f"  Linear:   |π₁ - π₂| = {abs(pi_small_linear_1 - pi_small_linear_2):.6f}")
print(f"  NonLinear: |π₁ - π₂| = {abs(pi_small_nonlinear_1 - pi_small_nonlinear_2):.6f}")

# Large difference
pi_large_linear_1 = linear_mixing(lambda_large_diff, phi_test)
pi_large_linear_2 = linear_mixing(lambda_large_diff_2, phi_test)
pi_large_nonlinear_1 = nonlinear_mixing(lambda_large_diff, phi_test)
pi_large_nonlinear_2 = nonlinear_mixing(lambda_large_diff_2, phi_test)

print(f"\nLarge Difference (λ₁: 5.0 → 5.5):")
print(f"  Linear:   |π₁ - π₂| = {abs(pi_large_linear_1 - pi_large_linear_2):.6f}")
print(f"  NonLinear: |π₁ - π₂| = {abs(pi_large_nonlinear_1 - pi_large_nonlinear_2):.6f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

scenarios = ['Small Diff\n(2.0→2.1)', 'Large Diff\n(5.0→5.5)']
linear_diffs = [abs(pi_small_linear_1 - pi_small_linear_2), abs(pi_large_linear_1 - pi_large_linear_2)]
nonlinear_diffs = [abs(pi_small_nonlinear_1 - pi_small_nonlinear_2), abs(pi_large_nonlinear_1 - pi_large_nonlinear_2)]

x = np.arange(len(scenarios))
width = 0.35

axes[0].bar(x - width/2, linear_diffs, width, label='Linear', alpha=0.8, color='steelblue')
axes[0].bar(x + width/2, nonlinear_diffs, width, label='Non-Linear', alpha=0.8, color='coral')
axes[0].set_xlabel('Scenario', fontsize=12)
axes[0].set_ylabel('|π₁ - π₂|', fontsize=12)
axes[0].set_title('Signal Preservation: Probability Difference', fontsize=13, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(scenarios)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3, axis='y')

# Show raw signal differences
raw_small = abs(lambda_small_diff[0] * phi_test[0] - lambda_small_diff_2[0] * phi_test[0])
raw_large = abs(lambda_large_diff[0] * phi_test[0] - lambda_large_diff_2[0] * phi_test[0])

axes[1].bar(x - width/2, [raw_small, raw_large], width, label='Raw Signal Diff', alpha=0.6, color='gray')
axes[1].bar(x + width/2, nonlinear_diffs, width, label='After Non-Linear Mix', alpha=0.8, color='coral')
axes[1].set_xlabel('Scenario', fontsize=12)
axes[1].set_ylabel('Difference', fontsize=12)
axes[1].set_title('Raw vs Processed Signal Differences', fontsize=13, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(scenarios)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('signal_preservation.png', dpi=150, bbox_inches='tight')
plt.show()


## 5. Why Aladynoulli Chose Linear Mixing

### Tradeoffs Summary:

| Aspect | Linear Mixing | Non-Linear Mixing |
|--------|---------------|-------------------|
| **Identifiability** | ✅ Better (softmax normalizes) | ❌ Scale invariant problem |
| **Separation** | ⚠️ Moderate (compression before mixing) | ✅ Better (raw mixing) |
| **Interpretability** | ✅ Normalized weights | ⚠️ Raw weights need normalization |
| **Stability** | ✅ More stable | ⚠️ Can be sensitive to scale |

### The Decision:

**Aladynoulli uses linear mixing** because:

1. **Identifiability is critical**: Without normalization, `λ` and `φ` are not uniquely identifiable
2. **Hot start with `ψ`**: The model initializes `γ` from `ψ_total`, which works better with normalized mixing
3. **Practical separation is sufficient**: The model achieves good discrimination despite compression before mixing
4. **Stability**: Normalized weights prevent numerical issues and make optimization more stable

### The Mathematical Insight:

With non-linear mixing:
```
sigmoid(λ × φ) = sigmoid((λ/c) × (φ×c))
```

This means you can't uniquely determine `λ` and `φ` - they're only identifiable up to a scale factor. Linear mixing avoids this by normalizing `λ` with `softmax`, making the model identifiable.


In [ ]:
# Final visualization: Side-by-side comparison
fig = plt.figure(figsize=(16, 10))

# Create a comprehensive comparison
lambda_test = np.array([3.0, 1.5, 0.5])
phi_test = np.array([2.0, 1.5, 1.0])

# Compute intermediate steps
lambda_probs = softmax(lambda_test)
phi_probs = sigmoid(phi_test)
linear_mixed = np.sum(lambda_probs * phi_probs)
linear_final = sigmoid(linear_mixed)

nonlinear_raw = np.sum(lambda_test * phi_test)
nonlinear_final = sigmoid(nonlinear_raw)

# Plot 1: Linear mixing steps
ax1 = plt.subplot(2, 2, 1)
steps_linear = ['λ (raw)', 'softmax(λ)', 'φ (raw)', 'sigmoid(φ)', 'Σ(softmax(λ)×sigmoid(φ))', 'sigmoid(Σ)']
values_linear = [lambda_test[0], lambda_probs[0], phi_test[0], phi_probs[0], linear_mixed, linear_final]
colors_linear = ['lightblue', 'steelblue', 'lightcoral', 'coral', 'lightgreen', 'green']
ax1.barh(steps_linear, values_linear, color=colors_linear, alpha=0.7)
ax1.set_xlabel('Value', fontsize=11)
ax1.set_title('Linear Mixing: Step-by-Step', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='x')

# Plot 2: Non-linear mixing steps
ax2 = plt.subplot(2, 2, 2)
steps_nonlinear = ['λ (raw)', 'φ (raw)', 'Σ(λ×φ)', 'sigmoid(Σ)']
values_nonlinear = [lambda_test[0], phi_test[0], nonlinear_raw, nonlinear_final]
colors_nonlinear = ['lightblue', 'lightcoral', 'lightgreen', 'green']
ax2.barh(steps_nonlinear, values_nonlinear, color=colors_nonlinear, alpha=0.7)
ax2.set_xlabel('Value', fontsize=11)
ax2.set_title('Non-Linear Mixing: Step-by-Step', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

# Plot 3: Scale invariance comparison
ax3 = plt.subplot(2, 2, 3)
scales = np.linspace(0.5, 3.0, 100)
pi_linear_scale = [linear_mixing(lambda_test/c, phi_test*c) for c in scales]
pi_nonlinear_scale = [nonlinear_mixing(lambda_test/c, phi_test*c) for c in scales]
ax3.plot(scales, pi_linear_scale, 'b-', linewidth=2, label='Linear', alpha=0.8)
ax3.plot(scales, pi_nonlinear_scale, 'r--', linewidth=2, label='Non-Linear', alpha=0.8)
ax3.axvline(x=1.0, color='gray', linestyle=':', alpha=0.5)
ax3.set_xlabel('Scale Factor c', fontsize=11)
ax3.set_ylabel('π(λ/c, φ×c)', fontsize=11)
ax3.set_title('Scale Invariance Test', fontsize=12, fontweight='bold')
ax3.legend(fontsize=10)
ax3.grid(True, alpha=0.3)

# Plot 4: Separation comparison
ax4 = plt.subplot(2, 2, 4)
lambda_range = np.linspace(0.5, 5, 100)
lambda_fixed = np.array([1.0, 1.0])
phi_fixed = np.array([2.0, 1.5])

pi_linear_sep = [linear_mixing(np.array([l, *lambda_fixed]), np.array([2.0, *phi_fixed])) for l in lambda_range]
pi_nonlinear_sep = [nonlinear_mixing(np.array([l, *lambda_fixed]), np.array([2.0, *phi_fixed])) for l in lambda_range]

ax4.plot(lambda_range, pi_linear_sep, 'b-', linewidth=2, label='Linear', alpha=0.8)
ax4.plot(lambda_range, pi_nonlinear_sep, 'r--', linewidth=2, label='Non-Linear', alpha=0.8)
ax4.set_xlabel('λ₁ (Signature 1 Loading)', fontsize=11)
ax4.set_ylabel('Disease Probability π', fontsize=11)
ax4.set_title('Separation: π vs λ₁', fontsize=12, fontweight='bold')
ax4.legend(fontsize=10)
ax4.grid(True, alpha=0.3)

plt.suptitle('Linear vs Non-Linear Mixing: Comprehensive Comparison', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('linear_vs_nonlinear_comprehensive.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"\nFinal Probabilities (λ={lambda_test}, φ={phi_test}):")
print(f"  Linear Mixing:    π = {linear_final:.6f}")
print(f"  Non-Linear Mixing: π = {nonlinear_final:.6f}")
print(f"\nKey Takeaway:")
print(f"  Linear mixing trades some separation potential for identifiability and stability.")
print(f"  This is the right choice for Aladynoulli's use case.")
